In [ ]:
from rateslib import *

# Valuing Historical Swaps at Today's Date

A common operation is to want to value a swap which has an *effective* date **in the past**.
To do this, the swap should not be altered in its definition but it likely requires additional pricing information supplied,
in the form of **fixings**. See *'Cookbook > Working with Fixings'* for more detail on this aspect.

In [ ]:
curve = Curve({dt(2024, 7, 3): 1.0, dt(2025, 7, 3): 0.95}, calendar="nyc")
irs = IRS(dt(2024, 6, 26), "1m", spec="usd_irs", curves=curve, fixed_rate=5.00)

This swap cant price directly because today is 3rd July and it started on 26th June and it is missing information on some of the fixings.

In [ ]:
try: 
    irs.npv()
except ValueError as e:
    print(e)

The SOFR fixings are needed for the following reference value dates in the past (as measured as of 3rd July 2024):

In [ ]:
get_calendar("nyc").bus_date_range(dt(2024, 6, 26), dt(2024, 7, 2))

Normally these fixings would be populated by some automated data collection service or CSV file upload, but here they have been manually inserted.

In [ ]:
from pandas import Series
fixings.add("sofr_fixings_1B", Series(
    data=[5.34, 5.34, 5.33, 5.40, 5.35],
    index=get_calendar("nyc").bus_date_range(dt(2024, 6, 26), dt(2024, 7, 2))
))
fixings["sofr_fixings_1B"]

The definition of the swap is altered to **also input the fixings** and then it is repriced with the curve, correctly, without raising errors.

In [ ]:
irs = IRS(dt(2024, 6, 26), "1m", spec="usd_irs", curves=curve, fixed_rate=5.0, leg2_rate_fixings="sofr_fixings")
irs.npv()

Alternatively the **cashflows** method gives an alternative, holistic perspective.

In [ ]:
irs.cashflows()

# Valuing Spot Swaps at Future Dates

The current design of *rateslib* targets accurate evaluation of instruments at today's date.
Valuing swaps as of future dates is something that might be required in calculations such as those for XVA or scenario analysis.

Currently *rateslib* does have some limited features to permit this.

As an example suppose today's date is 3rd July 2024 and we construct an 11m IRS with a monthly frequency. A more granular *Curve* is constructed in this example.

In [ ]:
curve = Curve(
    nodes={
        dt(2024, 7, 3): 1.0,
        dt(2024, 8, 3): 1.0,
        dt(2024, 9, 3): 1.0,
        dt(2024, 10, 3): 1.0,
        dt(2024, 11, 3): 1.0,
        dt(2024, 12, 3): 1.0,
        dt(2025, 1, 3): 1.0,
        dt(2025, 2, 3): 1.0,
        dt(2025, 3, 3): 1.0,
        dt(2025, 4, 3): 1.0,
        dt(2025, 5, 3): 1.0,
        dt(2025, 6, 3): 1.0,
        dt(2025, 7, 3): 1.0,
    },
    calendar="nyc",
)
solver = Solver(
    curves=[curve],
    instruments=[
        IRS(dt(2024, 7, 3), _, spec="usd_irs", curves=curve) for _ in
        ["1m", "2m", "3m", "4m", "5m", "6m", "7m", "8m", "9m", "10m", "11m", "1y"]
    ],
    s=[5.33, 5.33, 5.315, 5.28, 5.25, 5.21, 5.17, 5.15, 5.11, 5.07, 5.04, 5.00]
)

In [ ]:
irs = IRS(dt(2024, 7, 3), "11m", "M", spec="usd_irs", curves=curve, fixed_rate=5.04)
irs.npv()

### Today's date as the valuation evaluation date
Today's date, as the evaluation date, is derived from the initial node date of the **discount curve**, i.e. the date at which the discount factor is necessarily set to be equal to 1.0.

The initial node date can be **translated** on a *Curve* to a date in the future as follows.

In [ ]:
translated_curve = curve.translate(dt(2024, 10, 15))

If the swap is then attempted to be priced with this translated *Curve* it will fail for the same reason as the section above - it cannot forecast rates before it starts.

In [ ]:
try: 
    irs.npv(curves=translated_curve)
except ValueError as e:
    print(e)

However, in this case the fixings are not known because 15th October 2024 is a future date.
The solution is to supply a separate **forecast curve** and **discount curve**, where the forecast curve is capable of calculating the relevant rates.
In this case the forecast curve is the very same curve that is created as of today's date.


In [ ]:
irs.npv(curves=[curve, translated_curve])

When the cashflows are analysed, one can visually inspect that any cashflow that is paid before the initial node date of the discount curve is set to zero value. The floating rates are demonstarted here showing that the floating rates have still been correctly forecast for each *FloatPeriod*.

In [ ]:
irs.cashflows(curves=[curve, translated_curve])[12:16]

### Plotting Future Value

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1)

def _npv(date):
    t_curve = curve.translate(date)
    return irs.npv(curves=[curve, t_curve])
    
dates=[
    dt(2024, 7, 4),
    dt(2024, 8, 10),
    dt(2024, 9, 10),
    dt(2024, 10, 10),
    dt(2024, 11, 10),
    dt(2024, 12, 10),
    dt(2025, 1, 10),
    dt(2025, 2, 10),
    dt(2025, 3, 10),
    dt(2025, 4, 10),
    dt(2025, 5, 10),
    dt(2025, 6, 10),
]

ax.plot(dates, [_npv(_) for _ in dates])

The general structure of this plot is as described in the section on *'Cash, Collateral and Credit'* in **Pricing and Trading Interest Rate Derivatives**.